# SETUP AND USING TENSORFLOW GPU

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from os import listdir
from os.path import isfile, join
import random
import shutil
from matplotlib.image import imread

from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
import time
import seaborn as sns
import pathlib


# import DCGAN as gan

import json

from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, Activation, BatchNormalization, Lambda, MaxPooling2D,SpatialDropout2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

from pathlib import Path
import PIL
from keras.utils.vis_utils import plot_model
from keras_visualizer import visualizer 

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

In [4]:
IMG_DIR = './PokemonData'

In [5]:
def create_train_val_test_folder():
    os.system("mkdir train")
    os.system("mkdir test")
    os.system("mkdir val")
    
def clean_images():
    os.system("rm -r train/* test/* val/*")
    os.system("find train/ -name '*.*' -type f -delete")
    os.system("find val/ -name '*.*' -type f -delete")
    os.system("find test/ -name '*.*' -type f -delete")

def data_extractor(class_pct, train_pct, val_pct, test_pct):
    dirs = os.listdir(IMG_DIR)
    sub_classes = random.sample(dirs,int(len(dirs)*class_pct))

    for cla in sub_classes:
        os.system('mkdir "train/'+cla+'"')
        os.system('mkdir "test/'+cla+'"')
        os.system('mkdir "val/'+cla+'"')
        os.system("find train/"+cla+" -name '*.*' -type f -delete")
        os.system("find val/"+cla+" -name '*.*' -type f -delete")
        os.system("find test/"+cla+" -name '*.*' -type f -delete")

        temp_files = os.listdir(os.path.join(IMG_DIR,cla))

        files = [f for f in temp_files if isfile(os.path.join(IMG_DIR, cla,f))]
        
        train_files = random.sample(files,int(len(files)*(train_pct+val_pct+test_pct)))
        val_files = random.sample(train_files,int(len(files)*(val_pct+test_pct)))
        test_files = random.sample(val_files,int(len(files)*test_pct))
        train_files = [x for x in train_files if x not in val_files]
        val_files = [x for x in val_files if x not in test_files]
        
        for file in train_files:
            shutil.copy(os.path.join(IMG_DIR,cla,file), os.path.join('train',cla,file))
        for file in val_files:
            shutil.copy(os.path.join(IMG_DIR,cla,file), os.path.join('val',cla,file))
        for file in test_files:
            shutil.copy(os.path.join(IMG_DIR,cla,file), os.path.join('test',cla,file))

In [6]:
SampleSize= 1
TrainSize = 0.7
TestSize = 0.2
ValSize= 0.1

In [7]:
#data_extractor(SampleSize, TrainSize, ValSize, TestSize)

# Data preprocess

In [8]:
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [192, 192])
  image /= 255.0  # normalize to [0,1] range

  return image

def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  return preprocess_image(image)

In [9]:
data_root = Path('./train')
Test_root = Path('./test')
val_root = Path('./val')

In [10]:
import random

def load_and_preprocess_from_path_label(path, label):
  return load_and_preprocess_image(path), label

def create_dataset (root):
    all_image_paths = list(root.glob('*/*.jpg'))
    all_image_paths = [str(path) for path in all_image_paths]
    random.shuffle(all_image_paths)

    image_count = len(all_image_paths)

    print("The image count is %d" % image_count)
    label_names = sorted(item.name for item in root.glob('*/') if item.is_dir())

    label_to_index = dict((name, index) for index, name in enumerate(label_names))

    all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                        for path in all_image_paths]

    print("First 10 labels indices: ", all_image_labels[:10])

    steps_per_epoch=tf.math.ceil(len(all_image_paths)/BATCH_SIZE).numpy()
    print(steps_per_epoch)

    paths_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
    image_ds = paths_ds.map(load_and_preprocess_image)

    label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
    image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))

    ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))
    image_label_ds = ds.map(load_and_preprocess_from_path_label)
    

    ds = image_label_ds.shuffle(buffer_size=image_count)
    ds = ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

    return ds

In [11]:
train_ds = create_dataset(data_root)
val_ds = create_dataset(val_root)

The image count is 4733
First 10 labels indices:  [4, 50, 137, 75, 39, 103, 137, 41, 9, 146]
148.0
The image count is 653
First 10 labels indices:  [115, 129, 110, 50, 134, 76, 55, 84, 121, 105]
21.0


In [12]:
EPOCH = 50
image_shape = (192,192,3)
ClassNum = 150

In [13]:
def build_baseline():
    model = Sequential()

    model.add(Conv2D(filters=16,kernel_size=(3,3),padding='same',input_shape=image_shape,activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))

    model.add(Conv2D(filters=32,kernel_size=(3,3),padding='same',input_shape=image_shape,activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    
    model.add(Conv2D(filters=32,kernel_size=(3,3),padding='same',input_shape=image_shape,activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))

    model.add(Flatten())

    model.add(Dense(64,activation='relu'))

    model.add(Dense(16,activation='relu'))

    model.add(Dense(150,activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

    return model

In [14]:
def build_model_with_decay(initial_rate, decay_step, decay_rate):
    model = Sequential()

    model.add(Conv2D(filters=16,kernel_size=(3,3),padding='same',input_shape=image_shape,activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))

    model.add(Conv2D(filters=32,kernel_size=(3,3),padding='same',input_shape=image_shape,activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    
    model.add(Conv2D(filters=32,kernel_size=(3,3),padding='same',input_shape=image_shape,activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))

    model.add(Flatten())

    model.add(Dense(64,activation='relu'))

    model.add(Dense(16,activation='relu'))

    model.add(Dense(150,activation='softmax'))

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_rate,
    decay_steps=decay_step,
    decay_rate=decay_rate,
    staircase=True)


    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=lr_schedule), metrics=['accuracy'])

    return model

In [15]:
def compared_decay_rate():
    initials = [0.001,0.0001]
    steps = [10000,100000]
    rates = [0.96]
    historys =[]
    for initial in initials:
        for step in steps:
            for rate in rates:
                model = build_model_with_decay(initial,step,rate)
                history = model.fit(train_ds, epochs=EPOCH, validation_data = val_ds)
                historys.append(history)
                loss_history = pd.DataFrame(history.history)
                plt.plot(loss_history['val_accuracy'],label="initial: "+str(initial)+ " steps: "+str(step)+ " rate: "+str(rate))
    plt.legend()
    plt.show()
    plt.savefig("compared_decay_rate_"+time.strftime("%Y%m%d-%H%M%S")+".png")
    return historys

In [16]:
initial = 0.001,
step = 100
rate = 0.96
model = build_model_with_decay(initial,step,rate)
history = model.fit(train_ds, epochs=EPOCH, validation_data = val_ds)
json.dump(history.history,open("./historys/compare_lr_decay_"+str(initial)+"_"+str(step)+"_"+str(rate),'w'))

Epoch 1/50


ValueError: in user code:

    File "C:\Users\15197\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\15197\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\15197\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\15197\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 809, in train_step
        loss = self.compiled_loss(
    File "C:\Users\15197\AppData\Roaming\Python\Python39\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\Users\15197\AppData\Roaming\Python\Python39\site-packages\keras\losses.py", line